# INPATIENT METRICS
___

The goal of this project is to create dashboards that show the **average daily census**, **admissions**, **admissions by age**, **discharges**, **length of stay**, **counties of admission**, **admitted diagnoses**, and **waitlist information** for each month by each of the inpatient facilities. The tables are queried from the ODBCs and cleaned for analysis.

These are the metrics that are on each dashboard

|Metric|Definition|
|------|----------|
|Average Daily Census |The average number of pateints staying in the hospital per day                       |
|Admissions           |The number of patients admitted into a hospital                                      |
|Admissions by Age    |The number of patients grouped in their respective age category by their admitted age|
|Admitted Diagnoses   |The number of patients that are admitted with each particular diagnosis              |
|Counties of Admission|The number of patients admitted from their original county of residence              |
|Discharges           |The number of patient discharged from the hosital                                    |
|Length of Stay       |The number of days a patient is in the hospital starting from the admission date     |
|Waitlist             |The number of patients that are waiting to be admitted into a hospital               |

The following python packages below must be imported.

In [1]:
# Import python packages
import pyodbc
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib as plt
import datetime as dt
from datetime import timedelta
from datetime import date
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline

# Imports for Dash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Import for dealing with images
import plotly.io as pio

import os

# Change how many columns are viewed
# pd.set_option('display.max_columns', 1000000)
# pd.set_option('display.max_rows', 1000000)

# ODBC drivers that are avaialable
#pyodbc.drivers()

# If conda is giving an error when installing packages
# conda config --set ssl_verify no

# ODBC CONNECTIONS
_____

The **pyodbc** package is used to access the ODBC databases.


## Example #

**This is an example of connecting to an ODBC database**

driver = 'DRIVER NAME'  
server = 'SERVER NAME'  
host = 'HOTST NAME'  
port = 'PORT'  
database = 'DATABASE NAME'  
username = 'USERNAME'  
password = 'PASSWORD'  

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';HOST='+host+';PORT='+port+';DATABASE='+database+';UID='+username+';PWD='+ password)  
**cursor** = cnxn.cursor() - **The **cursor** is the variable that holds the database cursor and maps to the particular ODBC that is being connected.**

In [2]:
# Data sources that are avaialable in each ODBC
# pyodbc.dataSources()

**CAN NOT DISPLAY ODBC INFO**

**Print The Table Names**

This displays all the table names in the odbc

**Example:**

for i in [**odbc**(**cursor**)].tables():  
    print(i.table_name)

**Print the columns in each table**

This diplays all the columns in a table 

**Example:**

for i in [**odbc**(**cursor**)].columns(table='[**table_name**]'):  
print(i.column_name)

# Data
___

Once the ODBC connectons are established the data can be queried into the following tables and cleaned for analysis:

* **Admissions**
* **Discharges**
* **Census**
* **Current_Inpatients**
* **WaitList**

A SQL query is written and stored in the respective variables (**admissions, discharges, etc**). Then, the **cursor.executefunction** is used to retrieve the result set from the query against the ODBC Database.

#### Cleaning Each Table
In order get the information from each table for each month, the important dates such as admissions, discharges, and census will be formatted in a month-year format. A new column will be added to each necessary table called **[table name]_[monthyear]**. Some of the tables also have missing values where **null = NOT ENTERED IN SYSTEM**. Any null values will be replaced with **UNKNOWN**. These are two of the few ways that will be used to clean the tables.

# Date
_______
All of the data except for **Current_Inpatients** and **WaitList** needs to be filtered between speific dates. The **begin_date** and **end_date** will allow the data to be queried between these dates. The Date must be entered in the YYYY/MM-DD format. 

In [6]:
begin_date = '2020-07-01'
end_date = '2021-05-31'

### Admissions

* **admission_data**: The admissions dataset uncleaned 
* **admission_datac**: Copy of the admission_data cleaned

In [7]:
admissions  = ('''SELECT
                      patient_current_demographics.country_of_origin_value,
                      view_episode_summary_admit.admit_age_at,
                      view_episode_summary_admit.program_code,
                      view_episode_summary_admit.preadmit_admission_date,
                      view_episode_summary_admit.admit_principal_diag_code,
                      view_episode_summary_admit.admit_principal_diag_value
                  FROM view_episode_summary_admit
                  INNER JOIN patient_current_demographics ON patient_current_demographics.PATID = view_episode_summary_admit.PATID
                  WHERE view_episode_summary_admit.preadmit_admission_date BETWEEN '{0}' AND '{1}' '''
              ).format(begin_date, end_date)

admission_data = pd.DataFrame.from_records(avatar.execute(admissions).fetchall(), columns=['county_of_origin', 'adm_age', 
                                                                                         'program', 'admission_date', 'diag_code', 'diag_value'])
admission_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   county_of_origin  1567 non-null   object
 1   adm_age           1567 non-null   object
 2   program           1567 non-null   object
 3   admission_date    1567 non-null   object
 4   diag_code         1241 non-null   object
 5   diag_value        1241 non-null   object
dtypes: object(6)
memory usage: 73.6+ KB


**Clean Admission Data**

In [8]:
# Copy of the original data
admission_datac = admission_data.copy()

# Clean Null values
admission_datac['diag_code'] = admission_datac['diag_code'].fillna('UNKNOWN')
admission_datac['diag_value'] = admission_datac['diag_value'].fillna('UNKNOWN')

# Convert adm_age to number
admission_datac['adm_age'] = admission_datac['adm_age'].astype('int')

# Change admission dates to only month-year format
admission_datac['admission_monthyear'] = admission_datac['admission_date'].apply(lambda date: date.strftime('%b-%Y'))
# admission_datac['admission_monthyear'] = admission_datac['admission_monthyear'].apply(lambda date: dt.datetime.strptime(date, '%b-%Y'))

# Add an age category
for i, r in admission_datac.iterrows():
    
    age = r['adm_age']
    
    if age < 30:
        admission_datac.at[i, 'adm_age_cat'] = '<30'
    elif 30 <= age <= 49:
        admission_datac.at[i, 'adm_age_cat'] = '30-49'
    elif 50 <= age <= 64:
        admission_datac.at[i, 'adm_age_cat'] = '50-64'
    elif 65 <= age <= 79:
        admission_datac.at[i, 'adm_age_cat'] = '65-79'
    else:
        admission_datac.at[i, 'adm_age_cat'] = '80+'
        
admission_datac.head()

,county_of_origin,adm_age,program,admission_date,diag_code,diag_value,admission_monthyear,adm_age_cat
0,CHARLESTON,62,BPHFOR,2021-04-26,F02.81,Major neurocognitive disorder due to multiple ...,Apr-2021,50-64
1,RICHLAND,37,BPHAP,2020-12-04,F31.2,Severe manic bipolar 1 disorder with psychotic...,Dec-2020,30-49
2,BERKELEY,52,PADM,2021-03-08,UNKNOWN,UNKNOWN,Mar-2021,50-64
3,CHARLESTON,34,BPHFOR,2020-12-22,F20.9,Schizophrenia,Dec-2020,30-49
4,BAMBURG,33,BPHFOR,2021-04-21,F20.9,Schizophrenia,Apr-2021,30-49


In [9]:
admission_datac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   county_of_origin     1567 non-null   object
 1   adm_age              1567 non-null   int32 
 2   program              1567 non-null   object
 3   admission_date       1567 non-null   object
 4   diag_code            1567 non-null   object
 5   diag_value           1567 non-null   object
 6   admission_monthyear  1567 non-null   object
 7   adm_age_cat          1567 non-null   object
dtypes: int32(1), object(7)
memory usage: 91.9+ KB


### Discharges

Discharge types that have the code **EXPIRED** are excluded.

* **discharge_data**: The discharge dataset uncleaned 
* **discharge_datac**: Copy of the discharge_data cleaned

In [10]:
discharges = ('''SELECT
                      view_episode_summary_discharge.program_code,
                      view_episode_summary_discharge.date_of_discharge,
                      view_episode_summary_discharge.disc_type_of_value
                 FROM view_episode_summary_discharge
                 WHERE view_episode_summary_discharge.date_of_discharge BETWEEN '{0}' AND '{1}'
                 AND view_episode_summary_discharge.disc_type_of_value NOT IN('EXPIRED') '''
             ).format(begin_date, end_date)

discharge_data = pd.DataFrame.from_records(avatar.execute(discharges).fetchall(), columns=['program', 'discharge_date', 'discharge_type'])
discharge_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1431 entries, 0 to 1430
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   program         1431 non-null   object
 1   discharge_date  1431 non-null   object
 2   discharge_type  1431 non-null   object
dtypes: object(3)
memory usage: 33.7+ KB


**Clean discharges data**

In [11]:
# Copy of the original data
discharge_datac = discharge_data.copy()

# Change discharge dates to only month-year format and create new column
discharge_datac['discharge_monthyear'] = discharge_datac['discharge_date'].apply(lambda date: date.strftime('%b-%Y'))
discharge_datac.head()

,program,discharge_date,discharge_type,discharge_monthyear
0,GENER,2020-08-12,TRANSFER TO COURT/LAW ENFORCEMENT,Aug-2020
1,GENER,2021-03-02,ROUTINE DISCHARGE,Mar-2021
2,BPHFOR,2020-08-31,ROUTINE DISCHARGE,Aug-2020
3,BPHAP,2021-01-29,ROUTINE DISCHARGE,Jan-2021
4,PADM,2021-03-08,"PRE-ADMITTED, NOT ADMITTED",Mar-2021


### Census

* **census_data**: The census dataset uncleaned 
* **census_datac**: Copy of the census_data cleaned

In [12]:
census = ('''SELECT
                  billing_room_board_worklist.worklist_date,
                  billing_room_board_worklist.room_board_workist_program_code
             FROM billing_room_board_worklist
             INNER JOIN view_episode_summary_admit ON (view_episode_summary_admit.PATID = billing_room_board_worklist.PATID)
             AND (view_episode_summary_admit.EPISODE_NUMBER = billing_room_board_worklist.EPISODE_NUMBER)
             WHERE billing_room_board_worklist.worklist_date BETWEEN '{0}' AND '{1}' '''
         ).format(begin_date, end_date)

census_data = pd.DataFrame.from_records(avatar.execute(census).fetchall(), columns=['worklist_date', 'program'])
census_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369938 entries, 0 to 369937
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   worklist_date  369938 non-null  object
 1   program        369938 non-null  object
dtypes: object(2)
memory usage: 5.6+ MB


**Clean census data**

In [13]:
# Change census date to month-year format and create new column 
census_datac = census_data.copy()
census_datac['worklist_monthyear'] = census_datac['worklist_date'].apply(lambda date: date.strftime('%m%Y'))
census_datac['worklist_monthyear'] = census_datac['worklist_monthyear'].apply(lambda date: dt.datetime.strptime(date, '%m%Y'))
census_datac.head()

,worklist_date,program,worklist_monthyear
0,2020-07-01,BDTP,2020-07-01
1,2020-07-02,BDTP,2020-07-01
2,2020-07-03,BDTP,2020-07-01
3,2020-07-04,BDTP,2020-07-01
4,2020-07-05,BDTP,2020-07-01


### Current Inpatients

* **currinpat_data**: The census dataset uncleaned 
* **currinpat_datac**: Copy of the census_data cleaned

In [14]:
current_inpatients = ('''SELECT
                              view_episode_summary_current.preadmit_admission_date,
                              view_episode_summary_current.program_code
                         FROM view_episode_summary_current '''
                     )

currinpat_data = pd.DataFrame.from_records(avatar.execute(current_inpatients).fetchall(), columns=['admission_date', 'program'])
currinpat_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1299 entries, 0 to 1298
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   admission_date  1299 non-null   object
 1   program         1299 non-null   object
dtypes: object(2)
memory usage: 20.4+ KB


**Clean current inpatients data**

In [15]:
# Copy of the original data
currinpat_datac = currinpat_data.copy()

# Get the current day
today = dt.date(dt.datetime.today().year,
                dt.datetime.today().month,
                dt.datetime.today().day)

# Get the current length of Stay for each patient and add the column to the dataframe (Today's date - admission date)
currinpat_datac['los'] = currinpat_datac['admission_date'].apply(lambda admission_date: (today - admission_date).days)

# Create a LOS category column
currinpat_datac['los_category'] = ''

# LOS of stay categories
for i, r in currinpat_datac.iterrows():
    
    los = r['los']
    
    if los < 7:
        currinpat_datac.at[i, 'los_category'] = 'Less Than a Week'
    elif 7 <= los <= 20:
        currinpat_datac.at[i, 'los_category'] = '1 Week - 2 Weeks'
    elif 21 <= los <= 59:
        currinpat_datac.at[i, 'los_category'] = '3 Weeks - 1 Month' 
    elif 60 <= los <= 211:
        currinpat_datac.at[i, 'los_category'] = '2 Months - 6 Months' 
    elif 212 <= los <= 729:
        currinpat_datac.at[i, 'los_category'] = '7 Months - 1 Year'
    elif 730 <= los <= 2189:
        currinpat_datac.at[i, 'los_category'] = '2-5 Years'
    elif 2190 <= los <= 4014:
        currinpat_datac.at[i, 'los_category'] = '6-10 Years'
    elif 4015 <= los <= 7664:
        currinpat_datac.at[i, 'los_category'] = '11-20 Years'
    else:
        currinpat_datac.at[i, 'los_category'] = 'Over 20 Years'
        
currinpat_datac.head()

,admission_date,program,los,los_category
0,2018-06-28,BDTP,1072,2-5 Years
1,2018-11-14,BPHFOR,933,2-5 Years
2,2021-04-26,BPHFOR,39,3 Weeks - 1 Month
3,2008-04-16,BDTP,4797,11-20 Years
4,2016-08-13,BPHAP,1756,2-5 Years


### Wait List

* **waitlist_data**: The census dataset uncleaned 
* **waitlist_datac**: Copy of the census_data cleaned

In [16]:
waitlist = ('''SELECT
                    referral_1.dmhfacil,
                    ref_agency.ra_county,
                    county.descr,
                    referral_1.ref_date,
                    referral_2.typeadm,
                    adm_type.descr,
                    referral_2.emerg_admtype,
                    emerg_adm_type.descr, 
                    referral_1.status
              FROM referral_1
              INNER JOIN referral_2 ON (referral_2.patid = referral_1.patid)
              AND (referral_2.episode_num = referral_1.episode_num)
              INNER JOIN referral_walkin ON referral_walkin.ref_patid = referral_1.patid
              INNER JOIN adm_type ON adm_type.code = referral_2.typeadm
              INNER JOIN ref_agency ON ref_agency.ra_num = referral_2.ref_agency
              LEFT JOIN county ON ref_agency.ra_county = county.code
              LEFT JOIN emerg_adm_type ON referral_2.emerg_admtype = emerg_adm_type.code
              WHERE referral_1.status = 1''')

waitlist_data = pd.DataFrame.from_records(ip_emr.execute(waitlist).fetchall(), columns=['program', 'ref_agency_county', 
                                                                                      'ref_agency_county_descr', 'ref_date', 'ref_adm_type', 'adm_type_descr', 
                                                                                      'emerg_adm_type','emerg_adm_type_descr', 'ref_status'
                                                                                     ]
                                         )
waitlist_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   program                  415 non-null    object
 1   ref_agency_county        415 non-null    object
 2   ref_agency_county_descr  374 non-null    object
 3   ref_date                 415 non-null    object
 4   ref_adm_type             415 non-null    object
 5   adm_type_descr           415 non-null    object
 6   emerg_adm_type           414 non-null    object
 7   emerg_adm_type_descr     248 non-null    object
 8   ref_status               415 non-null    object
dtypes: object(9)
memory usage: 29.3+ KB


The columns **ref_agency_county_descr** and **emerg_adm_type_descr** are missing values

In [17]:
waitlist_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   program                  415 non-null    object
 1   ref_agency_county        415 non-null    object
 2   ref_agency_county_descr  374 non-null    object
 3   ref_date                 415 non-null    object
 4   ref_adm_type             415 non-null    object
 5   adm_type_descr           415 non-null    object
 6   emerg_adm_type           414 non-null    object
 7   emerg_adm_type_descr     248 non-null    object
 8   ref_status               415 non-null    object
dtypes: object(9)
memory usage: 29.3+ KB


**Clean Waitlist Data**

In [18]:
# Copy of the original waitlist_data
waitlist_datac = waitlist_data.copy()

# Convert ref_date to datetime
waitlist_datac['ref_date'] = pd.to_datetime(waitlist_datac['ref_date'])

# Clean the extra space in columns 1-3
for i in waitlist_datac.columns[0:3]:
    waitlist_datac[i] = waitlist_datac[i].str.strip()

# Clean extra spaces in columns 5+
for i in waitlist_datac.columns[5:]:
    waitlist_datac[i] = waitlist_datac[i].str.strip()
    
# Clean the null values
waitlist_datac['ref_agency_county_descr'] = waitlist_datac['ref_agency_county_descr'].fillna('UNKNOWN')
waitlist_datac['emerg_adm_type_descr'] = waitlist_datac['emerg_adm_type_descr'].fillna('UNKNOWN')
waitlist_datac.head()

,program,ref_agency_county,ref_agency_county_descr,ref_date,ref_adm_type,adm_type_descr,emerg_adm_type,emerg_adm_type_descr,ref_status
0,MV,22,GEORGETOWN,2009-03-25,JA,Judicial Admission,None,UNKNOWN,1
1,ERS,,UNKNOWN,2018-02-06,VO,Voluntary,,UNKNOWN,1
2,ERS,,UNKNOWN,2018-09-28,VO,Voluntary,,UNKNOWN,1
3,ERS,,UNKNOWN,2019-10-07,VO,Voluntary,,UNKNOWN,1
4,ERS,,UNKNOWN,2018-11-28,VO,Voluntary,,UNKNOWN,1


# Analysis
___

After the tables have been queried and cleaned they are now ready for analysis

##### Variables

These are static variables that will be used throughout the anlysis for the dashboards

|Variable|Description|
|--------|-----------|
|program|The inpatient facility codes|
|begin date|The beginning date the data is filtered for in the queries|
|end date|The ending date the data is filtered for in the queries|
|date|The unique list of dates in month-year format between the begin date and end date|
|date_range|The unique list of dates in month-year format between the begin date and end date as a string|
|todays_date|The date the script is ran as a string|

In [19]:
# Program codes for each facility
program = ['BPHAP', 'BPHFOR', 'BPHAR', 'BPHCA', 'HPHAP', 'MV', 'VVH', 'TNCC', 'CNCC', 'TCROD']

# Convert the speified begin date to datetime object
begin_date = dt.datetime.strptime(begin_date, '%Y-%m-%d')

# Convert the speified end date to datetime object
end_date = dt.datetime.strptime(end_date, '%Y-%m-%d')
delta = end_date - begin_date

date = []

# Creates a list of dates between the specified date range
for i in range(delta.days + 1):
    date.append(begin_date + timedelta(days=i))

# Convert the list of dates to a dataframe and format in Month-Year and get the unique values
date = pd.DataFrame(date)
date = date[0].apply(lambda date: date.strftime('%b-%Y'))
date = date.unique()

# Format the data date range specified in the date section above to a string
date_range = (str(begin_date.month) + '/' + str(begin_date.day) + '/' + str(begin_date.year) + ' - ' +
              str(end_date.month) + '/' + str(end_date.day) + '/' + str(end_date.year))

# format today's date to a string
todays_date = str(dt.datetime.today().month) + '/' + str(dt.datetime.today().day) + '/' + str(dt.datetime.today().year)

#### Total Admissions and Discharges per Program

When a resident is admitted into a facility the date they were admitted is stored; and when a resident leaves a facility the date of discharge is recorded. 
The table **ad_program** contains the counts for the total number of admissions and dischrges for each of the inpaitient facilities

In [20]:
# Create Dataframe to get the number of admissions per program
adm_program = pd.DataFrame(index=program, columns=['admissions']).fillna(0)

# Create Dataframe to get the number of discharges per program
disch_program = pd.DataFrame(index=program, columns=['discharges']).fillna(0)

# Count the number of admissions by program 
for i, r in admission_datac.iterrows():
    if r['program'] in program:
        adm_program.loc[r['program']] += 1
        
# Count the number of discharges by program 
for i, r in discharge_datac.iterrows():
    if r['program'] in program:
        disch_program.loc[r['program']] += 1

# Admissions and discharges all programs
ad_program = pd.concat([adm_program, disch_program], axis=1)
ad_program.loc['HALL'] = ad_program.loc['BPHAR'] + ad_program.loc['BPHCA']
ad_program

,admissions,discharges
BPHAP,83,100
BPHFOR,176,170
BPHAR,11,15
BPHCA,238,235
HPHAP,96,109
MV,527,515
VVH,88,8
TNCC,15,8
CNCC,130,35
TCROD,0,13


#### Admissions by Program by Month

The table **adm_pro_month** contains the total amount of admissions for each inpatient facility by month

In [21]:
# Create Dataframe
adm_pro_month = pd.DataFrame(index=date, columns=program).fillna(0)

# Count the number of admissins by program by month 
for i, r in admission_datac.iterrows():
    if r['program'] in program:
        adm_pro_month.loc[r['admission_monthyear'], r['program']] += 1

# Admissions for all program by month
adm_pro_month.index.name = 'Date'
adm_pro_month['HALL'] = adm_pro_month['BPHAR'] + adm_pro_month['BPHCA']
adm_pro_month

,BPHAP,BPHFOR,BPHAR,BPHCA,HPHAP,MV,VVH,TNCC,CNCC,TCROD,HALL
Date,,,,,,,,,,,
Jul-2020,6,2,5,26,9,34,1,0,0,0,31
Aug-2020,5,5,3,25,11,42,12,1,12,0,28
Sep-2020,6,23,3,25,6,44,12,1,13,0,28
Oct-2020,0,10,0,30,10,44,6,0,19,0,30
Nov-2020,0,13,0,16,5,42,10,2,10,0,16
Dec-2020,10,29,0,12,4,18,4,2,9,0,12
Jan-2021,12,2,0,14,4,53,0,2,2,0,14
Feb-2021,17,14,0,22,10,52,6,0,17,0,22
Mar-2021,8,31,0,21,9,75,13,3,23,0,21


#### Counties of Admission

The previous county of residence before being admitted into a facility is recorded. The **adm_counties** table contains the count of admissions by county previous ciunty of residence

In [22]:
# Create Dataframe to get values
c_value = pd.Series(admission_datac['county_of_origin'].unique()).sort_values().values # All counties in aplphabetical order
adm_counties = pd.DataFrame(index=c_value, columns=program).fillna(0)
adm_counties.index.name = 'County'

# Count of clients from each county that were admitted
for i, r in admission_datac.iterrows():
    if r['program'] in program:
        adm_counties.loc[r['county_of_origin'], r['program']] += 1

# Admissions by county for all programs
adm_counties['HALL'] = adm_counties['BPHAR'] + adm_counties['BPHCA']
adm_counties

,BPHAP,BPHFOR,BPHAR,BPHCA,HPHAP,MV,VVH,TNCC,CNCC,TCROD,HALL
County,,,,,,,,,,,
ABBEVILLE,0,1,0,2,3,4,0,0,2,0,2
AIKEN,0,7,0,0,0,7,0,1,1,0,0
ALLENDALE,0,1,0,0,0,1,0,0,0,0,0
ANDERSON,0,6,0,9,25,7,0,0,42,0,9
BAMBURG,0,1,0,1,0,1,1,0,0,0,1
BARNWELL,0,1,0,2,0,3,1,0,0,0,2
BEAUFORT,1,1,0,8,0,9,11,0,0,0,8
BERKELEY,3,8,0,0,0,9,11,0,1,0,0
CALHOUN,0,0,0,1,0,4,1,0,0,0,1


#### Admissions by Age

The table **adm_age** shows an age range for each facility based on the age of the resident on the date of admission

In [23]:
# Create Dataframe
prog_list = ad_program.index
prog_list.name = None
adm_age = pd.DataFrame(index=['<30', '30-49', '50-64', '65-79', '80+'], columns=program).fillna(0)

for i, r in admission_datac.iterrows():
    if r['program'] in program:
        adm_age.loc[r['adm_age_cat'], r['program']] += 1

adm_age['HALL'] = adm_age['BPHAR'] + adm_age['BPHCA']
adm_age

,BPHAP,BPHFOR,BPHAR,BPHCA,HPHAP,MV,VVH,TNCC,CNCC,TCROD,HALL
<30,23,46,11,238,27,131,0,0,0,0,249
30-49,38,99,0,0,45,284,0,0,0,0,0
50-64,18,22,0,0,21,102,4,1,5,0,0
65-79,4,9,0,0,3,10,30,1,41,0,0
80+,0,0,0,0,0,0,54,13,84,0,0


#### Admitted Diagnoses

The table **adm_diag** contains the count of primary diagnoses on the date of admission for each facility

In [24]:
# Create DataFrame to get values
adm_diag = pd.DataFrame(index=admission_datac['diag_value'].unique(), columns=program).fillna(0)

for i, r in admission_datac.iterrows():
    if r['program'] in program:
        adm_diag.loc[r['diag_value'], r['program']] += 1

adm_diag['HALL'] = adm_diag['BPHAR'] + adm_diag['BPHCA']
adm_diag

,BPHAP,BPHFOR,BPHAR,BPHCA,HPHAP,MV,VVH,TNCC,CNCC,TCROD,HALL
Major neurocognitive disorder due to multiple etiologies with behavioral disturbance,0,2,0,0,0,0,0,0,0,0,0
Severe manic bipolar 1 disorder with psychotic behavior,1,0,0,0,0,0,0,0,0,0,0
UNKNOWN,1,5,0,0,0,7,88,0,23,0,0
Schizophrenia,18,25,0,2,7,0,0,0,0,0,2
Unspecified schizophrenia spectrum and other psychotic disorder,2,22,0,4,6,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...
Attention deficit disorder,0,0,0,1,0,0,0,0,0,0,1
Right femoral fracture,0,0,0,0,0,0,0,1,0,0,0
Myasthenia gravis without (acute) exacerbation,0,0,0,0,0,0,0,1,0,0,0
"Schizophrenia, paranoid, chronic",0,0,0,0,1,0,0,0,0,0,0


#### Discharges by Program by Month

When a resident leaves a facility the date of discharge is recorded. The table **disch_pro_month** contains the count of dishcarges from each facility by month

In [25]:
# Create dataframe
disch_pro_month = pd.DataFrame(index=date, columns=program).fillna(0)

# Discharges by program by month
for i, r in discharge_datac.iterrows():
    if r['program'] in program:
        disch_pro_month.loc[r['discharge_monthyear'], r['program']] += 1
    
disch_pro_month.index.name = 'Date'
disch_pro_month['HALL'] = disch_pro_month['BPHAR'] + disch_pro_month['BPHCA']
disch_pro_month

,BPHAP,BPHFOR,BPHAR,BPHCA,HPHAP,MV,VVH,TNCC,CNCC,TCROD,HALL
Date,,,,,,,,,,,
Jul-2020,4,4,5,28,16,47,0,1,1,0,33
Aug-2020,9,10,4,25,10,28,1,0,9,0,29
Sep-2020,13,4,3,24,6,47,1,2,1,0,27
Oct-2020,9,18,3,27,11,48,0,1,5,1,30
Nov-2020,5,25,0,18,7,35,2,0,2,0,18
Dec-2020,13,12,0,16,9,40,0,2,0,3,16
Jan-2021,4,13,0,8,5,28,1,2,5,2,8
Feb-2021,8,23,0,22,3,48,2,0,0,0,22
Mar-2021,14,10,0,17,11,62,1,0,6,5,17


### Average Daily Census

While a resident is staying at any of the inpaitent facilities. Each day that the resident stays is recorded in the table. The **average_daily_census** contains the average amount of residents that were staying in each of the facilities per day for each month.

In [26]:
# Total number of patients for each day
census_subset_day = (census_datac.groupby(by='worklist_date', axis=0)
                            .agg('count')
                            .drop('program', axis=1)
                            .rename(columns={'cid': 'patient_count'})
                            .reset_index())

# Total days in each month
days = pd.DataFrame(index=census_datac['worklist_date'].unique(), columns=['days_in_month']).fillna(1).reset_index()
days['index'] = (days['index'].apply(lambda date: date.strftime('%b-%Y'))
                              .apply(lambda date: dt.datetime.strptime(date, '%b-%Y'))
                              .sort_values())
days = days.rename(columns={'index': 'month'}).groupby('month').agg('count')
days.index = days.index.strftime('%b-%Y')

# Days unique vlaues
days_dict = days.to_dict()['days_in_month']

# Total number of patients for each month
census_subset_month = census_subset_day
census_subset_month['worklist_date'] = (census_subset_month['worklist_date'].apply(lambda date: date.strftime('%b-%Y'))
                                                                            .apply(lambda date: dt.datetime.strptime(date, '%b-%Y')))
census_subset_month = census_subset_month.groupby('worklist_date').agg('sum')
census_subset_month.index = census_subset_month.index.strftime('%b-%Y')
census_subset_month.index.name = 'date'

# Average Daily Census (total patients/total days in month)
average_daily_census = census_subset_month.rename({census_subset_month.columns[0]: 'adc'}, axis=1)
average_daily_census['adc'] = [round(r[0]/days_dict[i]) for i, r in average_daily_census.iterrows()]
average_daily_census

,adc
date,
Jul-2020,1277
Aug-2020,1243
Sep-2020,1238
Oct-2020,1242
Nov-2020,1217
Dec-2020,1205
Jan-2021,1179
Feb-2021,1155
Mar-2021,1198


#### Average Daily Census by Month by Porgam

In [27]:
# Dataframe containing program and year
adc_pro_month_subset = census_datac[['program', 'worklist_monthyear']].copy()
adc_pro_month_subset['worklist_monthyear'] = adc_pro_month_subset['worklist_monthyear'].apply(lambda date: date.strftime('%b-%Y')) # convert the date back to string from datetime object
adc_pro_month = pd.DataFrame(index=days.index, 
                             columns=program
                            ).fillna(0)

# Total number of patients for each month by program
for i, r in adc_pro_month_subset.iterrows():
    if r['program'] in program:
        adc_pro_month.loc[r['worklist_monthyear'], r['program']] += 1
    
# Average Daily Census per month by program
for i, r in adc_pro_month.iterrows():
    adc_pro_month.loc[i] = round(r/days_dict[i])
    
adc_pro_month = adc_pro_month.astype('int')
adc_pro_month['HALL'] = adc_pro_month['BPHAR'] + adc_pro_month['BPHCA']
adc_pro_month

,BPHAP,BPHFOR,BPHAR,BPHCA,HPHAP,MV,VVH,TNCC,CNCC,TCROD,HALL
month,,,,,,,,,,,
Jul-2020,133,155,4,18,105,31,185,76,190,157,22
Aug-2020,129,149,5,17,100,31,182,73,175,155,22
Sep-2020,126,153,2,16,102,34,188,72,167,154,18
Oct-2020,116,165,1,19,99,38,189,67,172,152,20
Nov-2020,107,158,0,13,97,37,190,64,180,150,13
Dec-2020,105,156,0,18,95,26,191,61,184,149,18
Jan-2021,107,158,0,13,91,31,178,59,180,142,13
Feb-2021,114,142,0,21,94,42,160,56,174,135,21
Mar-2021,115,157,0,23,98,55,159,56,185,132,23


#### Length of Stay for Current Inpatients by Program

The LOS is the length a patient stays in the facility. This will sort the number of patients in each of the specified LOS categories. 

In [28]:
# Dataframe with program and los category
los = pd.DataFrame(index=['Less Than a Week', '1 Week - 2 Weeks', '3 Weeks - 1 Month', '2 Months - 6 Months',
                          '7 Months - 1 Year', '2-5 Years', '6-10 Years', '11-20 Years', 'Over 20 Years'],
                   columns=program
                  ).fillna(0)

# Total number of patients by los by program
for i, r in currinpat_datac.iterrows():
    if r['program'] in program:
        los.loc[r['los_category'], r['program']] += 1

# Add name to index
los.index.name = 'Length of Stay'
los['HALL'] = los['BPHAR'] + los['BPHCA']
los

,BPHAP,BPHFOR,BPHAR,BPHCA,HPHAP,MV,VVH,TNCC,CNCC,TCROD,HALL
Length of Stay,,,,,,,,,,,
Less Than a Week,1,6,0,6,3,13,3,1,3,0,6
1 Week - 2 Weeks,5,15,0,8,8,23,8,1,7,0,8
3 Weeks - 1 Month,10,30,0,9,10,15,14,2,11,0,9
2 Months - 6 Months,23,25,0,0,10,0,24,6,40,0,0
7 Months - 1 Year,21,31,0,0,12,1,48,12,52,6,0
2-5 Years,28,37,0,0,14,0,56,26,75,49,0
6-10 Years,16,11,0,0,16,0,19,7,8,28,0
11-20 Years,8,10,0,0,17,0,7,1,0,36,0
Over 20 Years,0,2,0,0,0,0,0,0,1,5,0


#### Patients Currently on the Wait List

Before a patient is admitted that are placed on a waiting list. This is a count of how many are on the waiting list.

##### Waitlist Variables

In [29]:
adm_type = waitlist_datac['adm_type_descr'].unique() # All admission types
pat_locat = waitlist_datac['emerg_adm_type_descr'].unique() # All patient locations

In [30]:
# Create dataframe to get the count of patients on waitlist
waitlist_count = pd.DataFrame(index=program, columns=['waitlist total']).fillna(0)

# Count of patients currently on the waitlist per facility
for i, r in waitlist_datac.iterrows():
    if r['program'] in program:
        waitlist_count.loc[r['program']] += 1
    
waitlist_count.loc['HALL'] = waitlist_count.loc['BPHAR'] + waitlist_count.loc['BPHCA']
waitlist_count

,waitlist total
BPHAP,78
BPHFOR,98
BPHAR,0
BPHCA,2
HPHAP,10
MV,162
VVH,5
TNCC,0
CNCC,19
TCROD,32


In [31]:
# Wait List Categories by program
waitlist_cat = pd.DataFrame(index=adm_type, columns=program).fillna(0)

for i, r in waitlist_datac.iterrows():
    if r['program'] in program:
        waitlist_cat.loc[r['adm_type_descr'], r['program']] += 1
        
waitlist_cat['HALL'] = waitlist_cat['BPHAR'] + waitlist_cat['BPHCA']
waitlist_cat

,BPHAP,BPHFOR,BPHAR,BPHCA,HPHAP,MV,VVH,TNCC,CNCC,TCROD,HALL
Judicial Admission,35,28,0,0,3,22,0,0,0,0,0
Voluntary,0,0,0,0,0,138,5,0,19,32,0
Emergency Admission,43,7,0,2,7,1,0,0,0,0,2
Restoration,0,62,0,0,0,1,0,0,0,0,0
Not Guilty by Reason,0,1,0,0,0,0,0,0,0,0,0


In [32]:
# Wait list Location by Program
waitlist_loc = pd.DataFrame(index=pat_locat, columns=program).fillna(0)

for i, r in waitlist_datac.iterrows():
    if r['program'] in program:
        waitlist_loc.loc[r['emerg_adm_type_descr'], r['program']] += 1
        
waitlist_loc['HALL'] = waitlist_loc['BPHAR'] + waitlist_loc['BPHCA']
waitlist_loc

,BPHAP,BPHFOR,BPHAR,BPHCA,HPHAP,MV,VVH,TNCC,CNCC,TCROD,HALL
UNKNOWN,5,90,0,0,0,7,5,0,19,32,0
HOME,14,0,0,0,0,142,0,0,0,0,0
JAIL,15,7,0,0,1,3,0,0,0,0,0
PSYCH FACILITY,11,1,0,0,2,4,0,0,0,0,0
A & D Facility,0,0,0,0,0,4,0,0,0,0,0
ER,31,0,0,2,6,2,0,0,0,0,2
ER - ADMITTED,2,0,0,0,1,0,0,0,0,0,0


# PLOTS

The dashboards that are mentioned in the introduction are created by separating each facility into its own dashboard

## Hospital Metrics Plots

This creates dashboads for the hospitals:
* **BPHAP**
* **BPHFOR**
* **HALL**
* **HPHAP**

In [33]:
hosp = ['BPHAP', 'BPHFOR', 'HALL', 'HPHAP', 'MV'] # codes for the hospitals
hosp_titles = {'BPHAP': 'FACILITY 1',
          'BPHFOR': 'FACILITY 2',
          'HALL': 'FACILITY 3',
          'HPHAP': 'FACILITY 4',
          'MV': 'FACILITY 5'
         } # dictonary of titles for each of the facilities

In [34]:
for prog_code in hosp:

    fig = make_subplots(rows=3, cols=3,
                        subplot_titles=['<b>Average Daily Census</b>', '<b>Admissions</b>', '<b>Discharges</b>',
                                        '<b>Length of Stay</b>', '<b>Counties of Admission</b>', '<b>Admitted Diagnoses</b>',
                                        '<b>Waiting List Categories</b>', '<b>Patient Location</b>', '<b>Total on Waiting List, {0}</b>'.format(todays_date)],
                        specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}],
                               [{'type': 'bar'}, {'type': 'pie'}, {'type': 'pie'}],
                               [{'type': 'bar'}, {'type': 'bar'}, {'type': 'domain'}]]
                       )
    
    # Set the font size
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=25)

    # Average Daily Census
    fig.add_trace(go.Bar(x=adc_pro_month[prog_code].index, 
                         y=adc_pro_month[prog_code],
                         text=adc_pro_month[prog_code],
                         textposition='outside',
                         ), row=1, col=1
                 )
    # Update The y-axis
    fig.update_yaxes(range=[0, max(adc_pro_month[prog_code].values)+15],
                     row=1,
                     col=1)

    # Admissions 
    fig.add_trace(go.Bar(x=adm_pro_month[prog_code].index, 
                         y=adm_pro_month[prog_code], 
                         text=adm_pro_month[prog_code],
                         marker=dict(color='lightgreen'),
                         textposition='outside'
                         ), row=1, col=2
                 )
    # Update The y-axis
    fig.update_yaxes(range=[0, max(adm_pro_month[prog_code].values)+10],
                     row=1,
                     col=2)

    # Discharges 
    fig.add_trace(go.Bar(x=disch_pro_month[prog_code].index, 
                         y=disch_pro_month[prog_code], 
                         text=disch_pro_month[prog_code],
                         marker=dict(color='salmon'),
                         textposition='outside'
                         ), row=1, col=3
                 )
    # Update The y-axis
    fig.update_yaxes(range=[0, max(disch_pro_month[prog_code].values)+10],
                     row=1,
                     col=3
                    )

    # Length of Stay 
    fig.add_trace(go.Bar(x=los[prog_code],
                         y=los[prog_code].index,
                         width = 1,
                         text=los[prog_code],
                         marker=dict(color='green'),
                         textposition='outside',
                         orientation='h'
                         ), row=2, col=1
                 )

    # Counties of Admission
    fig.add_trace(go.Pie(labels=adm_counties[prog_code].sort_values(ascending=False).index[:5],
                         values=adm_counties[prog_code].sort_values(ascending=False).values[:5],
                         textinfo = 'label+percent',
                         textfont_size = 25,
                         textposition='inside'
                         ), row=2, col=2
                 )

    # Admitted Diagnoses
    # Convert numbers to percentage
    p = pd.DataFrame(adm_diag[prog_code].sort_values(ascending=False))
    p = p[p[prog_code] != 0]
    p = p.apply(lambda n: round((n/ad_program.loc[prog_code, 'admissions'])*100, 2))

    fig.add_trace(go.Table(header=dict(values=['<b>Diagnoses</b>', '<b>%</b>'], 
                                       align='center',
                                       height=40,
                                       fill_color='skyblue'),
                           cells=dict(values=[p.index, p], 
                                      align='center',
                                      height=40,
                                      font=dict(size=20),
                                      fill_color='lightblue'),
                          ), row=2, col=3)

    # Waiting List Categories
    fig.add_trace(go.Bar(x=waitlist_cat[prog_code].index,
                         y=waitlist_cat[prog_code], 
                         text=waitlist_cat[prog_code],
                         marker=dict(color='gold'),
                         textposition='outside',
                         ), row=3, col=1
                 )
    # Update The y-axis
    fig.update_yaxes(range=[0, max(waitlist_cat[prog_code].values)+10],
                     row=3,
                     col=1
                    )

    # Patient Location
    fig.add_trace(go.Bar(x=waitlist_loc[prog_code].index,
                         y=waitlist_loc[prog_code], 
                         text=waitlist_loc[prog_code],
                         marker=dict(color='royalblue'),
                         textposition='outside',
                         ), row=3, col=2
                 )
    # Update The y-axis
    fig.update_yaxes(range=[0, max(waitlist_loc[prog_code].values)+10],
                     row=3,
                     col=2
                    )

    # Total Patients on waiting list
    fig.add_trace(go.Indicator(mode = "number+gauge",
                               gauge = {'shape': 'bullet',
                                        'axis': {'visible': True}},
                               value = waitlist_count.loc[prog_code][0],
                               domain = {'x': [0.1, 1], 'y': [0.2, 0.9]},
#                              title = {'text': "Total"}
                              ), row=3, col=3
                 )

    fig.update_layout(height=1800, 
                      width=3000, 
                      title_text="<b>{0}          {1}</b>".format(hosp_titles[prog_code],
                                                                  date_range),
                      font=dict(size=25),
                      title_font=dict(size=30),
                      title_x=0.5,
                      showlegend=False)

    # Write the image to pdf
    fig.write_image("C:/Users/DDR33/Software/Jupyter/Dashboards/{0} - Metrics.pdf".format(prog_code)) # Write the figure to a file

# Long Term Care Metrics Plots

This creates dashboads for the long term care facilities:
* **VVH**
* **TNCC**
* **CNCC**
* **TCROD**

In [35]:
ltc = ['VVH', 'TNCC', 'CNCC', 'TCROD'] # codes for the long term care facilities
ltc_titles = {'VVH': 'FACILITY 6',
              'TNCC': 'FACILITY 7',
              'CNCC': 'FACILITY 8',
              'TCROD': 'FACILITY 9'
             } # dictonary of titles for each of the facilities

In [36]:
for prog_code in ltc:
    fig = make_subplots(rows=3, cols=4,
                        subplot_titles=['<b>Average Daily Census</b>', '<b>Admissions</b>', '<b>Admissions by Age</b>', '<b>Length of Stay</b>',
                                        '<b>Discharges</b>', '<b>Counties of Admission</b>', '<b>Admitted Diagnoses</b>',
                                        '<b>Waiting List Categories</b>', '<b>Patient Location</b>', '<b>Total on Waiting List, {0}</b>'.format(todays_date)],
                        specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}],
                               [{'type': 'bar'},{'type': 'pie', 'colspan': 2}, None, {'type': 'pie'}],
                               [{'type': 'bar', 'colspan':2}, None, {'type': 'bar'}, {'type': 'domain'}]]
                       )
    
    # Set the font size
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=25)


    # Average Daily Census
    fig.add_trace(go.Bar(x=adc_pro_month[prog_code].index, 
                         y=adc_pro_month[prog_code],
                         text=adc_pro_month[prog_code], 
                         textposition='outside'
                         ), row=1, col=1
                 )
    # Update The y-axis
    fig.update_yaxes(range=[0, max(adc_pro_month[prog_code].values)+15],
                     row=1,
                     col=1)

    # Admissions 
    fig.add_trace(go.Bar(x=adm_pro_month[prog_code].index, 
                         y=adm_pro_month[prog_code], 
                         text=adm_pro_month[prog_code],
                         marker=dict(color='lightgreen'),
                         textposition='outside'
                         ), row=1, col=2
                 )
    # Update The y-axis
    fig.update_yaxes(range=[0, max(adm_pro_month[prog_code].values)+10],
                     row=1,
                     col=2)

    # Admissions by Age
    fig.add_trace(go.Bar(x=adm_age[prog_code].index, 
                         y=adm_age[prog_code], 
                         text=adm_age[prog_code],
                         marker=dict(color='lightgreen'),
                         textposition='outside'
                         ), row=1, col=3
                 )
    # Update The y-axis
    fig.update_yaxes(range=[0, max(adm_age[prog_code].values)+10],
                     row=1,
                     col=3)

    # Length of Stay 
    fig.add_trace(go.Bar(x=los[prog_code].index,
                         y=los[prog_code], 
                         text=los[prog_code],
                         marker=dict(color='green'),
                         textposition='outside',
                         ), row=1, col=4
                 )
    # Update The y-axis
    fig.update_yaxes(range=[0, max(los[prog_code].values)+10],
                     row=1,
                     col=4)

    # Discharges 
    fig.add_trace(go.Bar(x=disch_pro_month[prog_code].index, 
                         y=disch_pro_month[prog_code], 
                         text=disch_pro_month[prog_code],
                         marker=dict(color='salmon'),
                         textposition='outside'
                         ), row=2, col=1
                 )
    # Update The y-axis
    fig.update_yaxes(range=[0, max(disch_pro_month[prog_code].values)+10],
                     row=2,
                     col=1)

    # Counties of Admission
    fig.add_trace(go.Pie(labels=adm_counties[prog_code].sort_values(ascending=False).index[:5],
                         values=adm_counties[prog_code].sort_values(ascending=False).values[:5],
                         textinfo = 'label+percent',
                         textfont_size = 25,
                         textposition='inside'
                         ), row=2, col=2
                 )

    # Admitted Diagnoses
    # Convert numbers to percentage
    p = pd.DataFrame(adm_diag[prog_code].sort_values(ascending=False))
    p = p[p[prog_code] != 0]
    p = p.apply(lambda n: round((n/ad_program.loc[prog_code, 'admissions'])*100, 2))

    fig.add_trace(go.Table(header=dict(values=['<b>Diagnoses</b>', '<b>%</b>'], 
                                       align='center',
                                       height=40,
                                       fill_color='skyblue'),
                           cells=dict(values=[p.index, p], 
                                      align='center',
                                      height=60,
                                      font=dict(size=20),
                                      fill_color='lightblue')
                          ), row=2, col=4)

    # Waiting List Categories
    fig.add_trace(go.Bar(x=waitlist_cat[prog_code],
                         y=waitlist_cat[prog_code].index, 
                         text=waitlist_cat[prog_code], 
                         textposition='outside',
                         marker=dict(color='gold'),
                         orientation='h'
                         ), row=3, col=1
                 )

    # Patient Location
    fig.add_trace(go.Bar(x=waitlist_loc[prog_code].index,
                         y=waitlist_loc[prog_code], 
                         text=waitlist_loc[prog_code], 
                         marker=dict(color='royalblue'),
                         textposition='outside',
                         ), row=3, col=3
                 )
    # Update The y-axis
    fig.update_yaxes(range=[0, max(waitlist_loc[prog_code].values)+10],
                     row=3,
                     col=3)

    # Patients on waiting list
    fig.add_trace(go.Indicator(mode = "number+gauge",
                               gauge = {'shape': 'bullet',
                                        'axis': {'visible': True}},
                               value = waitlist_count.loc[prog_code][0],
                               domain = {'x': [0.1, 1], 'y': [0.2, 0.9]},
#                                title = {'text': "Total"}
                              ), row=3, col=4
                 )

    fig.update_layout(height=1800, 
                      width=3100, 
                      title_text="<b>{0}          {1}</b>".format(ltc_titles[prog_code],
                                                                                                     date_range),
                      font=dict(size=21),
                      title_font=dict(size=30),
                      title_x = 0.5,
                      showlegend=False)
    
    fig.write_image("C:/Users/DDR33/Software/Jupyter/Dashboards/{0} - Metrics.pdf".format(prog_code)) # Write the figure to a file

# Summary Plots

In [37]:
dashboards = ['ADMISSIONS', 'AVERAGE DAILY CENSUS', 'LENGTH OF STAY', 'WAITLIST'] # file names the plots will be written to
programs = {'BPHAP': 'FACILITY 1',
            'BPHFOR': 'FACILITY 2',
            'HALL': 'FACILITY 3',
            'HPHAP': 'FACILITY 4',
            'MV': 'FACILITY 5',
            'VVH': 'FACILITY 6',
            'TNCC': 'FACILITY 7',
            'CNCC': 'FACILITY 8',
            'TCROD': 'FACILITY 9'}

# Number of rows for subplots
n_rows=3

# Number of columns for subplots
n_cols=3

In [38]:
for i in dashboards:
    
    # ADMISSIONS
    if i == 'ADMISSIONS':
        
        # Subplot titles
        subplot_titles = []
        for j in programs.values():
            subplot_titles.append('<b>{0} {1}</b>'.format(j, i))
            
        # Set the size of the figure
        fig = make_subplots(rows=n_rows, 
                            cols=n_cols,
                            subplot_titles=subplot_titles,
                            specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}],
                                   [{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}],
                                   [{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}]])
        
        # set the font size of the subplot titles
        for s in fig['layout']['annotations']:
            s['font'] = dict(size=25)
        
        # initialize the rows and clumns for each subplot    
        row = 1
        col = 0

        for k, v in enumerate(programs.keys()):
            col += 1
            
            # ADMISSIONS subplots
            fig.add_trace(go.Bar(x=adm_pro_month[v].index, 
                                 y=adm_pro_month[v], 
                                 text=adm_pro_month[v],
                                 marker=dict(color='lightgreen'),
                                 textposition='outside'
                                 ), row=row, col=col)
            
            # Update The y-axis for each subplot
            fig.update_yaxes(range=[0, max(adm_pro_month[v].values)+10],
                             row=row,
                             col=col)
            
            # resets column
            if col == n_cols:
                col = 0
                row += 1
  
        fig.update_layout(height=1800, 
                          width=3100, 
                          title_text="<b>{0}          {1}</b>".format(i,
                                                                                                date_range
                                                                                               ),
                          font=dict(size=25),
                          title_font=dict(size=30),
                          title_x=0.5,
                          showlegend=False)
    
    # AVERAGE DAILY CENSUS
    if i == 'AVERAGE DAILY CENSUS':
        
        # Subplot titles
        subplot_titles = []
        for j in programs.values():
            subplot_titles.append('<b>{0} {1}</b>'.format(j, i))
            
        # Set the size of the figure
        fig = make_subplots(rows=n_rows, 
                            cols=n_cols,
                            subplot_titles=subplot_titles,
                            specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}],
                                   [{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}],
                                   [{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}]])
        
        # set the font size of the subplot titles
        for s in fig['layout']['annotations']:
            s['font'] = dict(size=25)
        
        # initialize the rows and clumns for each subplot    
        row = 1
        col = 0

        for k, v in enumerate(programs.keys()):
            col += 1
            
            # AVERAGE DAILY CENSUS subplots
            fig.add_trace(go.Bar(x=adc_pro_month[v].index, 
                                 y=adc_pro_month[v], 
                                 text=adc_pro_month[v],
                                 marker=dict(color='royalblue'),
                                 textposition='outside'
                                 ), row=row, col=col)
            
            # Update The y-axis for each subplot
            fig.update_yaxes(range=[0, max(adc_pro_month[v].values)+25],
                             row=row,
                             col=col)
            
            # resets column
            if col == n_cols:
                col = 0
                row += 1
  
        fig.update_layout(height=1800, 
                          width=3100, 
                          title_text="<b>{0}          {1}</b>".format(i,
                                                                                                    date_range
                                                                                                   ),
                          font=dict(size=25),
                          title_font=dict(size=30),
                          title_x=0.5,
                          showlegend=False)
    
    # LENGTH OF STAY
    if i == 'LENGTH OF STAY':
        
        # Subplot titles
        subplot_titles = []
        for j in programs.values():
            subplot_titles.append('<b>{0} {1}</b>'.format(j, i))
            
        # Set the size of the figure
        fig = make_subplots(rows=n_rows, 
                            cols=n_cols,
                            subplot_titles=subplot_titles,
                            specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}],
                                   [{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}],
                                   [{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}]])
        
        # set the font size of the subplot titles
        for s in fig['layout']['annotations']:
            s['font'] = dict(size=25)
        
        # initialize the rows and clumns for each subplot    
        row = 1
        col = 0

        for k, v in enumerate(programs.keys()):
            col += 1
            
            # LENGTH OF STAY subplots
            fig.add_trace(go.Bar(x=los[v], 
                                 y=los[v].index, 
                                 text=los[v],
                                 marker=dict(color='green'),
                                 textposition='outside',
                                 orientation='h'
                                 ), row=row, col=col)
            
            # resets column
            if col == n_cols:
                col = 0
                row += 1
  
        fig.update_layout(height=1800, 
                          width=3100, 
                          title_text="<b>{0}          {1}</b>".format(i,
                                                                                                    date_range
                                                                                                   ),
                          font=dict(size=18),
                          title_font=dict(size=30),
                          title_x=0.5,
                          showlegend=False)
    
    fig.write_image("C:/Users/DDR33/Software/Jupyter/Dashboards/Summary Dashboards/{0}.pdf".format(i)) # Write the figure to a file
    
    # WAITLISTS
    if i == 'WAITLIST':
        
        # Subplot titles
        subplot_titles = []
        for j in programs.values():
            subplot_titles.append('<b>{0} {1}</b>'.format(j, i))
            
        # Set the size of the figure
        fig = make_subplots(rows=n_rows, 
                            cols=n_cols,
                            subplot_titles=subplot_titles,
                            specs=[[{'type': 'domain'}, {'type': 'domain'}, {'type': 'domain'}],
                                   [{'type': 'domain'}, {'type': 'domain'}, {'type': 'domain'}],
                                   [{'type': 'domain'}, {'type': 'domain'}, {'type': 'domain'}]])
        
        # set the font size of the subplot titles
        for s in fig['layout']['annotations']:
            s['font'] = dict(size=25)
        
        # initialize the rows and clumns for each subplot    
        row = 1
        col = 0

        for k, v in enumerate(programs.keys()):
            col += 1
            
            # WAITLISTS SUBPLOTS
            fig.add_trace(go.Indicator(mode = "number+gauge",
                               gauge = {'shape': 'bullet',
                                        'axis': {'visible': True}},
                               value = waitlist_count.loc[v][0],
                               domain = {'x': [0.1, 1], 'y': [0.2, 0.9]},
                              ), row=row, col=col
                 )
            
            # resets column
            if col == n_cols:
                col = 0
                row += 1
  
        fig.update_layout(height=1800, 
                      width=3100, 
                      title_text="<b>{0}          AS OF, {1}</b>".format(i,
                                                                                                todays_date
                                                                                               ),
                      font=dict(size=18),
                      title_font=dict(size=30),
                      title_x=0.5,
                      showlegend=False) 
    
        fig.write_image("C:/Users/DDR33/Software/Jupyter/Dashboards/Summary Dashboards/{0}.pdf".format(i)) # Write the figure to a file

# Close ODBC CONNECTIONS

In [39]:
avatar.close()
avatar_live.close()
ip_emr.close

<function Cursor.close>